## Feature Selection

In [ ]:
import os
import time
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

N_ITER_PER_BLOCK = 200
CV_SPLITS = 5
TEST_SIZE = 0.2

feature_cols = [
    "EURO_1", "EURO_2", "EURO_3", "EURO_4", "EURO_5", "EURO_6", "EURO_CLEAN",
    "Previous", "TotalFleet"
]
context_cols = ["CITY_AREA", "POPULATION", "Population_density"]
cars_per_surface = [
    "CARS_PER_KM2", "EURO_1_PER_KM2", "EURO_2_PER_KM2", "EURO_3_PER_KM2", "EURO_4_PER_KM2",
    "EURO_5_PER_KM2", "EURO_6_PER_KM2", "EURO_CLEAN_PER_KM2", "Previous_PER_KM2"
]
cars_per_capita = [
    "CARS_PER_CAPITA", "EURO_1_PER_CAPITA", "EURO_2_PER_CAPITA",
    "EURO_3_PER_CAPITA", "EURO_4_PER_CAPITA", "EURO_5_PER_CAPITA", "EURO_6_PER_CAPITA",
    "EURO_CLEAN_PER_CAPITA", "Previous_PER_CAPITA"
]

blocks = {
    'block_1': feature_cols,
    'block_2': feature_cols + context_cols,
    'block_3': feature_cols + context_cols + cars_per_capita,
    'block_4': feature_cols + cars_per_capita,
    'block_5': feature_cols + context_cols + cars_per_surface,
    'block_6': feature_cols + cars_per_surface,
    'block_7': feature_cols + context_cols + cars_per_surface + cars_per_capita,
    'block_8': feature_cols + cars_per_surface + cars_per_capita
}

outer_cv = KFold(CV_SPLITS, shuffle=True, random_state=42)

dataset_path = os.path.join('..', 'Data', 'Final_Dataset', 'final_dataset.parquet')
results_list = []

dataset = pl.read_parquet(dataset_path)
pollutants_cols = [col for col in dataset.columns if col.startswith('MONTHLY')]
raw = dataset.to_pandas()


fit_counter = 0

In [ ]:
for pollutant_col in pollutants_cols:
    pollutant = pollutant_col.split('_')[0][7:]
    raw_aux = raw[raw[pollutant_col].notna()].reset_index(drop=True)
    
    for name, cols in blocks.items():
        print(f'Tuning pollutant {pollutant} with {name} block')
        
        missing_cols = [col for col in cols if col not in raw_aux.columns]
        if missing_cols:
            print(f"Missing columns: {missing_cols}")
            available_cols = [col for col in cols if col in raw_aux.columns]
            if not available_cols:
                print(f"Skipping {name}: No valid columns")
                continue
            cols = available_cols
        
        X = raw_aux[cols]
        y = raw_aux[pollutant_col]
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=TEST_SIZE, random_state=42, shuffle=True
        )

        est = LGBMRegressor(
            objective='regression',
            metric='rmse',
            random_state=42,
            n_jobs=1,
            verbosity=-1,
            reg_alpha=0.1,
            reg_lambda=0.1,
        )
        
        param_dist = {
            'num_leaves': sp_randint(2, 66),
            'max_depth': sp_randint(3, 51),
            'learning_rate': sp_uniform(0.01, 0.15),
            'n_estimators': sp_randint(100, 601),
            'min_child_samples': sp_randint(1, 51),
            'reg_alpha': sp_uniform(0.0, 2.0),
            'reg_lambda': sp_uniform(0.0, 2.0),
            'colsample_bytree': sp_uniform(0.6, 0.4),
            'subsample': sp_uniform(0.6, 0.4),
            'min_split_gain': sp_uniform(0.0, 0.5),
            'boosting_type': ['gbdt']
        }
        
        search = RandomizedSearchCV(
            est, param_dist,
            n_iter=N_ITER_PER_BLOCK,
            cv=outer_cv,
            scoring='neg_root_mean_squared_error',
            random_state=1,
            n_jobs=-1,
            verbose=0
        )
        
        start = time.time()
        
        try:
            search.fit(X_train, y_train)
            elapsed = time.time() - start
            
            cv_rmse = -search.best_score_
            
            best_model = search.best_estimator_
            y_pred_test = best_model.predict(X_test)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
            
            overfitting_gap = test_rmse - cv_rmse
            overfitting_ratio = test_rmse / cv_rmse if cv_rmse > 0 else np.inf
            
            fits_this_search = N_ITER_PER_BLOCK * CV_SPLITS
            fit_counter += fits_this_search
            
            results_list.append({
                'pollutant': pollutant,
                'block': name,
                'cv_rmse': cv_rmse,
                'test_rmse': test_rmse,
                'overfitting_gap': overfitting_gap,
                'overfitting_ratio': overfitting_ratio,
                'best_params': search.best_params_,
                'duration_s': int(elapsed),
                'n_features': len(cols),
                'train_size': len(X_train),
                'test_size': len(X_test),
                'fits_performed': fits_this_search
            })
            
            print(f"[{pollutant}-{name}] CV rmse: {cv_rmse:.4f}, Test rmse : {test_rmse:.4f}")
            
        except Exception as e:
            print(f"Error: {e}")
            fits_this_search = N_ITER_PER_BLOCK * CV_SPLITS
            fit_counter += fits_this_search
            
            results_list.append({
                'pollutant': pollutant,
                'block': name,
                'cv_rmse': np.nan,
                'test_rmse': np.nan,
                'overfitting_gap': np.nan,
                'overfitting_ratio': np.nan,
                'best_params': None,
                'duration_s': int(time.time() - start),
                'n_features': len(cols),
                'train_size': len(X_train) if 'X_train' in locals() else 0,
                'test_size': len(X_test) if 'X_test' in locals() else 0,
                'fits_performed': fits_this_search,
                'error': str(e)
            })

df_results = pd.DataFrame(results_list)
output_path = os.path.join('..', 'Models', 'Results', '1st_stage.csv')
df_results.to_csv(output_path, index=False)

In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

output_path = os.path.join('..','Models','Results','1st_stage.csv')
feature_selections = pl.read_csv(output_path)

scored = (
    feature_selections.with_columns(
        (100 * (1 - (pl.col("test_rmse") - pl.col("test_rmse").min().over("pollutant"))
                         / pl.col("test_rmse").min().over("pollutant")))
        .alias("score")
    )
)

heat = (
    scored.pivot(values="score", index="pollutant", columns="block").sort("pollutant")
)

matrix = heat.drop("pollutant").to_numpy()

vmin, vmax = 90, 100
cmap = plt.get_cmap("Blues").copy()
cmap.set_under("lightgrey")
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)

fig, ax = plt.subplots(figsize=(matrix.shape[1]*1.4,
                                matrix.shape[0]*0.6 + 2))
im = ax.imshow(matrix, cmap=cmap, norm=norm)

ax.set_xticks(np.arange(matrix.shape[1]),
              labels=heat.columns[1:], rotation=45, ha="right")
ax.set_yticks(np.arange(matrix.shape[0]),
              labels=heat["pollutant"])
ax.set_xlabel("Feature block")
ax.set_ylabel("Pollutant")
ax.set_title("Percent improvement relative to the best feature-block combination \n(colour scale 80–100)")

for i, pollutant in enumerate(heat["pollutant"]):
    for j, block in enumerate(heat.columns[1:]):
        rmse = (
            feature_selections
            .filter((pl.col("pollutant") == pollutant) &
                    (pl.col("block") == block))
            .select("test_rmse")
            .item()
        )
        ax.text(j, i, f"{rmse:.3g}", ha="center", va="center",
                color="black", fontsize=8)
fig.colorbar(im, ax=ax, shrink=0.8, label="% of best (90–100)")
plt.tight_layout()
output_path = os.path.join('..','Figures','Model_feature_selec.png')
plt.savefig(output_path)
plt.show()

In [ ]:
import os
import polars as pl
output_path = os.path.join('..','Models','Results','1st_stage.csv')
feature_selections = pl.read_csv(output_path)

feature_selections.filter(pl.col('block') == 'per_capita_non_context').select(['pollutant', 'overfitting_ratio'])

## Hyper parameter tuning

### 1st iteration (2nd stage)

In [ ]:
import os
import time
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

N_ITER = 500
CV_SPLITS = 5
TEST_SIZE = 0.2

outer_cv = KFold(CV_SPLITS, shuffle=True, random_state=42)
dataset_path = os.path.join('..', 'Data', 'Final_Dataset', 'final_dataset.parquet')
output_path = os.path.join('..', 'Models', 'Results', '2nd_stage.csv')
results_list = []

dataset = pl.read_parquet(dataset_path)
pollutants_cols = [col for col in dataset.columns if col.startswith('MONTHLY')]
raw = dataset.to_pandas()

cols = [
    "EURO_1", "EURO_2", "EURO_3", "EURO_4", "EURO_5", "EURO_6", "EURO_CLEAN",
    "Previous", "TotalFleet", "CARS_PER_CAPITA", "EURO_1_PER_CAPITA", "EURO_2_PER_CAPITA",
    "EURO_3_PER_CAPITA", "EURO_4_PER_CAPITA", "EURO_5_PER_CAPITA", "EURO_6_PER_CAPITA",
    "EURO_CLEAN_PER_CAPITA", "Previous_PER_CAPITA"
]

param_dist = {
    'num_leaves': sp_randint(5, 50),
    'max_depth': sp_randint(3, 12),
    'learning_rate': sp_uniform(0.01, 0.09),
    'n_estimators': sp_randint(100, 500),
    'min_child_samples': sp_randint(15, 100),
    'reg_alpha': sp_uniform(0.1, 1.9),
    'reg_lambda': sp_uniform(0.1, 1.9),
    'colsample_bytree': sp_uniform(0.5, 0.4),
    'subsample': sp_uniform(0.6, 0.3),
    'min_split_gain': sp_uniform(0.1, 0.4),
    'boosting_type': ['gbdt']
}

fit_counter = 0

for pollutant_col in pollutants_cols:
    pollutant = pollutant_col.split('_')[0][7:]
    raw_aux = raw[raw[pollutant_col].notna()].reset_index(drop=True)
    
    print(f'tuning pollutant {pollutant}')
    
    missing_cols = [col for col in cols if col not in raw_aux.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")
        available_cols = [col for col in cols if col in raw_aux.columns]
        if not available_cols:
            print(f"Skipping {pollutant}: No valid columns")
            continue
        cols_to_use = available_cols
    else:
        cols_to_use = cols
    
    X = raw_aux[cols_to_use]
    y = raw_aux[pollutant_col]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42, shuffle=True
    )

    est = LGBMRegressor(
        objective='regression',
        metric='rmse',
        random_state=42,
        n_jobs=1,
        verbosity=-1,
        reg_alpha=0.2,
        reg_lambda=0.2,
        min_child_samples=20,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5
    )
    
    search = RandomizedSearchCV(
        est, param_dist,
        n_iter=N_ITER,
        cv=outer_cv,
        scoring='neg_root_mean_squared_error',
        random_state=1,
        n_jobs=-1,
        verbose=0
    )
    
    start = time.time()
    
    try:
        search.fit(X_train, y_train)
        elapsed = time.time() - start
        
        cv_rmse = -search.best_score_
    
        best_model = search.best_estimator_
        
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        overfitting_gap = test_rmse - cv_rmse
        overfitting_ratio = test_rmse / cv_rmse if cv_rmse > 0 else np.inf
        train_test_gap = test_rmse - train_rmse
        train_test_ratio = test_rmse / train_rmse if train_rmse > 0 else np.inf
        
        cv_flag = "Failed" if overfitting_ratio > 1.0 else "Passed"
        train_flag = "Failed" if train_test_ratio > 1.5 else "Passed"
        
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': cv_rmse,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'overfitting_gap': overfitting_gap,
            'overfitting_ratio': overfitting_ratio,
            'train_test_gap': train_test_gap,
            'train_test_ratio': train_test_ratio,
            'best_params': search.best_params_,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use),
            'train_size': len(X_train),
            'test_size': len(X_test),
            'fits_performed': N_ITER * CV_SPLITS
        })
        
        print(f"[{pollutant}] CV: {cv_rmse:.4f}, Train: {train_rmse:.4f}, Test: {test_rmse:.4f}")
        print(f"CV/Test: {overfitting_ratio:.3f} {cv_flag}, Train/Test: {train_test_ratio:.3f} {train_flag}")
        
    except Exception as e:
        print(f"Error tuning {pollutant}: {e}")
        elapsed = time.time() - start
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': np.nan,
            'train_rmse': np.nan,
            'test_rmse': np.nan,
            'overfitting_gap': np.nan,
            'overfitting_ratio': np.nan,
            'train_test_gap': np.nan,
            'train_test_ratio': np.nan,
            'best_params': None,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use) if 'cols_to_use' in locals() else len(cols),
            'train_size': len(X_train) if 'X_train' in locals() else 0,
            'test_size': len(X_test) if 'X_test' in locals() else 0,
            'fits_performed': N_ITER * CV_SPLITS,
            'error': str(e)
        })

df_results = pd.DataFrame(results_list)
df_results.to_csv(output_path, index=False)

#### CO and PM10 already passed

### 2nd iteration (3rd stage)

In [ ]:
a = pl.read_csv(os.path.join('..','Models','Results','2nd_stage.csv'))
feature_selections.filter(pl.col('block') == 'per_capita')['best_params'].to_list()
pollutants = a['pollutant'].unique().to_list()
for pollutant in pollutants:
    params = a.filter((pl.col('pollutant') == pollutant))['best_params'].item()
    print(f'For the pollutant {pollutant} the best parameters where:')
    print(f'{params}')

In [ ]:
import os
import time
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

N_ITER = 500
CV_SPLITS = 5
TEST_SIZE = 0.2

outer_cv = KFold(CV_SPLITS, shuffle=True, random_state=42)
dataset_path = os.path.join('..', 'Data', 'Final_Dataset', 'final_dataset.parquet')
output_path = os.path.join('..', 'Models', 'Results', '3rd_stage.csv')
output_path = os.path.join('..', 'Models', 'Results', '2nd_stage.csv')
results_list = []

dataset = pl.read_parquet(dataset_path)
pollutants_cols = [col for col in dataset.columns if col.startswith('MONTHLY')]
raw = dataset.to_pandas()

cols = [
    "EURO_1", "EURO_2", "EURO_3", "EURO_4", "EURO_5", "EURO_6", "EURO_CLEAN",
    "Previous", "TotalFleet", "CARS_PER_CAPITA", "EURO_1_PER_CAPITA", "EURO_2_PER_CAPITA",
    "EURO_3_PER_CAPITA", "EURO_4_PER_CAPITA", "EURO_5_PER_CAPITA", "EURO_6_PER_CAPITA",
    "EURO_CLEAN_PER_CAPITA", "Previous_PER_CAPITA"
]

param_dist = { 
    'num_leaves': sp_randint(15, 35),
    'max_depth': sp_randint(6, 11),
    'learning_rate': sp_uniform(0.01, 0.08),
    'n_estimators': sp_randint(100, 376),
    'min_child_samples': sp_randint(20, 51),
    'reg_alpha': sp_uniform(0.1, 1.2),
    'reg_lambda': sp_uniform(0.2, 1.8),
    'colsample_bytree': sp_uniform(0.5, 0.3),
    'subsample': sp_uniform(0.6, 0.3),
    'min_split_gain': sp_uniform(0.1, 0.2),
}

fit_counter = 0

for pollutant_col in pollutants_cols:
    pollutant = pollutant_col.split('_')[0][7:]
    raw_aux = raw[raw[pollutant_col].notna()].reset_index(drop=True)
    
    print(f'tuning pollutant {pollutant}')
    
    missing_cols = [col for col in cols if col not in raw_aux.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")
        available_cols = [col for col in cols if col in raw_aux.columns]
        if not available_cols:
            print(f"Skipping {pollutant}: No valid columns")
            continue
        cols_to_use = available_cols
    else:
        cols_to_use = cols
    
    X = raw_aux[cols_to_use]
    y = raw_aux[pollutant_col]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42, shuffle=True
    )

    est = LGBMRegressor(
        objective='regression',
        metric='rmse',
        random_state=42,
        n_jobs=1,
        verbosity=-1,
        reg_alpha=0.2,
        reg_lambda=0.2,
        min_child_samples=20,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5
    )
    
    search = RandomizedSearchCV(
        est, param_dist,
        n_iter=N_ITER,
        cv=outer_cv,
        scoring='neg_root_mean_squared_error',
        random_state=1,
        n_jobs=-1,
        verbose=0
    )
    
    start = time.time()
    
    try:
        search.fit(X_train, y_train)
        elapsed = time.time() - start
        
        cv_rmse = -search.best_score_
    
        best_model = search.best_estimator_
        
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        overfitting_gap = test_rmse - cv_rmse
        overfitting_ratio = test_rmse / cv_rmse if cv_rmse > 0 else np.inf
        train_test_gap = test_rmse - train_rmse
        train_test_ratio = test_rmse / train_rmse if train_rmse > 0 else np.inf
        
        cv_flag = "Failed" if overfitting_ratio > 1.0 else "Passed"
        train_flag = "Failed" if train_test_ratio > 1.5 else "Passed"
        
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': cv_rmse,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'overfitting_gap': overfitting_gap,
            'overfitting_ratio': overfitting_ratio,
            'train_test_gap': train_test_gap,
            'train_test_ratio': train_test_ratio,
            'best_params': search.best_params_,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use),
            'train_size': len(X_train),
            'test_size': len(X_test),
            'fits_performed': N_ITER * CV_SPLITS
        })
        
        print(f"[{pollutant}] CV: {cv_rmse:.4f}, Train: {train_rmse:.4f}, Test: {test_rmse:.4f}")
        print(f"CV/Test: {overfitting_ratio:.3f} {cv_flag}, Train/Test: {train_test_ratio:.3f} {train_flag}")
        
    except Exception as e:
        print(f"Error tuning {pollutant}: {e}")
        elapsed = time.time() - start
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': np.nan,
            'train_rmse': np.nan,
            'test_rmse': np.nan,
            'overfitting_gap': np.nan,
            'overfitting_ratio': np.nan,
            'train_test_gap': np.nan,
            'train_test_ratio': np.nan,
            'best_params': None,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use) if 'cols_to_use' in locals() else len(cols),
            'train_size': len(X_train) if 'X_train' in locals() else 0,
            'test_size': len(X_test) if 'X_test' in locals() else 0,
            'fits_performed': N_ITER * CV_SPLITS,
            'error': str(e)
        })

df_results = pd.DataFrame(results_list)
df_results.to_csv(output_path, index=False)

### Not enough for the rest of the models, NO2 more or less

### 3rd iteratiom (4th stage)

In [ ]:
a = pl.read_csv(os.path.join('..','Models','Results','2nd_stage.csv'))
feature_selections.filter(pl.col('block') == 'per_capita')['best_params'].to_list()
pollutants = a['pollutant'].unique().to_list()
pollutants = ['CO','PM10','NO2','PM25','O3']
for pollutant in pollutants:
    params = a.filter((pl.col('pollutant') == pollutant))['best_params'].item()
    print(f'For the pollutant {pollutant} the best parameters where:')
    print(f'{params}')

In [ ]:
import os
import time
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

N_ITER = 500
CV_SPLITS = 5
TEST_SIZE = 0.2

outer_cv = KFold(CV_SPLITS, shuffle=True, random_state=42)
dataset_path = os.path.join('..', 'Data', 'Final_Dataset', 'final_dataset.parquet')
output_path = os.path.join('..', 'Models', 'Results', '4th_stage.csv')
results_list = []

dataset = pl.read_parquet(dataset_path)
pollutants_cols = [col for col in dataset.columns if col.startswith('MONTHLY')]
raw = dataset.to_pandas()

cols = [
    "EURO_1", "EURO_2", "EURO_3", "EURO_4", "EURO_5", "EURO_6", "EURO_CLEAN",
    "Previous", "TotalFleet", "CARS_PER_CAPITA", "EURO_1_PER_CAPITA", "EURO_2_PER_CAPITA",
    "EURO_3_PER_CAPITA", "EURO_4_PER_CAPITA", "EURO_5_PER_CAPITA", "EURO_6_PER_CAPITA",
    "EURO_CLEAN_PER_CAPITA", "Previous_PER_CAPITA"
]

param_dist = {
    'num_leaves': sp_randint(18, 27),
    'max_depth': sp_randint(5, 10),
    'learning_rate': sp_uniform(0.02, 0.065),  
    'n_estimators': sp_randint(300, 360),
    'min_child_samples': sp_randint(20, 41),
    'reg_alpha': sp_uniform(0.33, 0.66),
    'reg_lambda': sp_uniform(0.5, 1),
    'colsample_bytree': sp_uniform(0.5, 0.20),
    'subsample': sp_uniform(0.6, 0.2),
    'min_split_gain': sp_uniform(0.10, 0.11),
    'boosting_type': ['gbdt']
}

fit_counter = 0

for pollutant_col in pollutants_cols:
    pollutant = pollutant_col.split('_')[0][7:]
    raw_aux = raw[raw[pollutant_col].notna()].reset_index(drop=True)
    
    print(f'tuning pollutant {pollutant}')
    
    missing_cols = [col for col in cols if col not in raw_aux.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")
        available_cols = [col for col in cols if col in raw_aux.columns]
        if not available_cols:
            print(f"Skipping {pollutant}: No valid columns")
            continue
        cols_to_use = available_cols
    else:
        cols_to_use = cols
    
    X = raw_aux[cols_to_use]
    y = raw_aux[pollutant_col]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42, shuffle=True
    )

    est = LGBMRegressor(
        objective='regression',
        metric='rmse',
        random_state=42,
        n_jobs=1,
        verbosity=-1,
        reg_alpha=0.2,
        reg_lambda=0.2,
        min_child_samples=20,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5
    )
    
    search = RandomizedSearchCV(
        est, param_dist,
        n_iter=N_ITER,
        cv=outer_cv,
        scoring='neg_root_mean_squared_error',
        random_state=1,
        n_jobs=-1,
        verbose=0
    )
    
    start = time.time()
    
    try:
        search.fit(X_train, y_train)
        elapsed = time.time() - start
        
        cv_rmse = -search.best_score_
    
        best_model = search.best_estimator_
        
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        overfitting_gap = test_rmse - cv_rmse
        overfitting_ratio = test_rmse / cv_rmse if cv_rmse > 0 else np.inf
        train_test_gap = test_rmse - train_rmse
        train_test_ratio = test_rmse / train_rmse if train_rmse > 0 else np.inf
        
        cv_flag = "Failed" if overfitting_ratio > 1.0 else "Passed"
        train_flag = "Failed" if train_test_ratio > 1.5 else "Passed"
        
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': cv_rmse,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'overfitting_gap': overfitting_gap,
            'overfitting_ratio': overfitting_ratio,
            'train_test_gap': train_test_gap,
            'train_test_ratio': train_test_ratio,
            'best_params': search.best_params_,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use),
            'train_size': len(X_train),
            'test_size': len(X_test),
            'fits_performed': N_ITER * CV_SPLITS
        })
        
        print(f"[{pollutant}] CV: {cv_rmse:.4f}, Train: {train_rmse:.4f}, Test: {test_rmse:.4f}")
        print(f"CV/Test: {overfitting_ratio:.3f} {cv_flag}, Train/Test: {train_test_ratio:.3f} {train_flag}")
        
    except Exception as e:
        print(f"Error tuning {pollutant}: {e}")
        elapsed = time.time() - start
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': np.nan,
            'train_rmse': np.nan,
            'test_rmse': np.nan,
            'overfitting_gap': np.nan,
            'overfitting_ratio': np.nan,
            'train_test_gap': np.nan,
            'train_test_ratio': np.nan,
            'best_params': None,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use) if 'cols_to_use' in locals() else len(cols),
            'train_size': len(X_train) if 'X_train' in locals() else 0,
            'test_size': len(X_test) if 'X_test' in locals() else 0,
            'fits_performed': N_ITER * CV_SPLITS,
            'error': str(e)
        })

df_results = pd.DataFrame(results_list)
df_results.to_csv(output_path, index=False)

### NO 2 passed

### 4th iteration (5th_stafe)

In [ ]:
a = pl.read_csv(os.path.join('..','Models','Results','3rd_stage.csv'))
feature_selections.filter(pl.col('block') == 'per_capita')['best_params'].to_list()
pollutants = a['pollutant'].unique().to_list()
pollutants = ['CO','PM10','NO2','PM25','O3']
for pollutant in pollutants:
    params = a.filter((pl.col('pollutant') == pollutant))['best_params'].item()
    print(f'For the pollutant {pollutant} the best parameters where:')
    print(f'{params}')

In [ ]:
import os
import time
import polars as pl
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint, uniform as sp_uniform
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error

N_ITER = 500
CV_SPLITS = 5
TEST_SIZE = 0.2

outer_cv = KFold(CV_SPLITS, shuffle=True, random_state=42)
dataset_path = os.path.join('..', 'Data', 'Final_Dataset', 'final_dataset.parquet')
output_path = os.path.join('..', 'Models', 'Results', '5th_stage.csv')
results_list = []

dataset = pl.read_parquet(dataset_path)
pollutants_cols = [col for col in dataset.columns if col.startswith('MONTHLY')]
raw = dataset.to_pandas()

cols = [
    "EURO_1", "EURO_2", "EURO_3", "EURO_4", "EURO_5", "EURO_6", "EURO_CLEAN",
    "Previous", "TotalFleet", "CARS_PER_CAPITA", "EURO_1_PER_CAPITA", "EURO_2_PER_CAPITA",
    "EURO_3_PER_CAPITA", "EURO_4_PER_CAPITA", "EURO_5_PER_CAPITA", "EURO_6_PER_CAPITA",
    "EURO_CLEAN_PER_CAPITA", "Previous_PER_CAPITA"
]

param_dist = {
    'colsample_bytree': sp_uniform(0.65, 0.14),
    'learning_rate': sp_uniform(0.01, 0.03),
    'max_depth': sp_randint(5, 8),
    'min_child_samples': sp_randint(18, 21),
    'min_split_gain': sp_uniform(0.15, 0.04),
    'n_estimators': sp_randint(310, 350),
    'num_leaves': sp_randint(24, 31),
    'reg_alpha': sp_uniform(0.1, 0.4),
    'reg_lambda': sp_uniform(0.9, 0.8),
    'subsample': sp_uniform(0.75, 0.20),
}

fit_counter = 0

for pollutant_col in pollutants_cols:
    pollutant = pollutant_col.split('_')[0][7:]
    raw_aux = raw[raw[pollutant_col].notna()].reset_index(drop=True)
    
    print(f'tuning pollutant {pollutant}')
    
    missing_cols = [col for col in cols if col not in raw_aux.columns]
    if missing_cols:
        print(f"Missing columns: {missing_cols}")
        available_cols = [col for col in cols if col in raw_aux.columns]
        if not available_cols:
            print(f"Skipping {pollutant}: No valid columns")
            continue
        cols_to_use = available_cols
    else:
        cols_to_use = cols
    
    X = raw_aux[cols_to_use]
    y = raw_aux[pollutant_col]
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=42, shuffle=True
    )

    est = LGBMRegressor(
        objective='regression',
        metric='rmse',
        random_state=42,
        n_jobs=1,
        verbosity=-1,
        reg_alpha=0.2,
        reg_lambda=0.2,
        min_child_samples=20,
        feature_fraction=0.8,
        bagging_fraction=0.8,
        bagging_freq=5
    )
    
    search = RandomizedSearchCV(
        est, param_dist,
        n_iter=N_ITER,
        cv=outer_cv,
        scoring='neg_root_mean_squared_error',
        random_state=1,
        n_jobs=-1,
        verbose=0
    )
    
    start = time.time()
    
    try:
        search.fit(X_train, y_train)
        elapsed = time.time() - start
        
        cv_rmse = -search.best_score_
    
        best_model = search.best_estimator_
        
        y_pred_train = best_model.predict(X_train)
        y_pred_test = best_model.predict(X_test)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        
        overfitting_gap = test_rmse - cv_rmse
        overfitting_ratio = test_rmse / cv_rmse if cv_rmse > 0 else np.inf
        train_test_gap = test_rmse - train_rmse
        train_test_ratio = test_rmse / train_rmse if train_rmse > 0 else np.inf
        
        cv_flag = "Failed" if overfitting_ratio > 1.0 else "Passed"
        train_flag = "Failed" if train_test_ratio > 1.5 else "Passed"
        
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': cv_rmse,
            'train_rmse': train_rmse,
            'test_rmse': test_rmse,
            'overfitting_gap': overfitting_gap,
            'overfitting_ratio': overfitting_ratio,
            'train_test_gap': train_test_gap,
            'train_test_ratio': train_test_ratio,
            'best_params': search.best_params_,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use),
            'train_size': len(X_train),
            'test_size': len(X_test),
            'fits_performed': N_ITER * CV_SPLITS
        })
        
        print(f"[{pollutant}] CV: {cv_rmse:.4f}, Train: {train_rmse:.4f}, Test: {test_rmse:.4f}")
        print(f"CV/Test: {overfitting_ratio:.3f} {cv_flag}, Train/Test: {train_test_ratio:.3f} {train_flag}")
        
    except Exception as e:
        print(f"Error tuning {pollutant}: {e}")
        elapsed = time.time() - start
        fit_counter += N_ITER * CV_SPLITS
        
        results_list.append({
            'pollutant': pollutant,
            'cv_rmse': np.nan,
            'train_rmse': np.nan,
            'test_rmse': np.nan,
            'overfitting_gap': np.nan,
            'overfitting_ratio': np.nan,
            'train_test_gap': np.nan,
            'train_test_ratio': np.nan,
            'best_params': None,
            'duration_s': int(elapsed),
            'n_features': len(cols_to_use) if 'cols_to_use' in locals() else len(cols),
            'train_size': len(X_train) if 'X_train' in locals() else 0,
            'test_size': len(X_test) if 'X_test' in locals() else 0,
            'fits_performed': N_ITER * CV_SPLITS,
            'error': str(e)
        })

df_results = pd.DataFrame(results_list)
df_results.to_csv(output_path, index=False)

### All the models passed

## Model training

In [ ]:
import os
import polars as pl 
stage2   = os.path.join('..', 'Models', 'Results', '2nd_stage.csv')
dataset = pl.read_csv(stage2).filter((pl.col('pollutant') == 'CO') |
                                     (pl.col('pollutant') == 'PM10'))

stage4   = os.path.join('..', 'Models', 'Results', '4th_stage.csv')
aux_dataset = pl.read_csv(stage4).filter(pl.col('pollutant')== 'NO2')
dataset = pl.concat([dataset,aux_dataset])

stage5   = os.path.join('..', 'Models', 'Results', '5th_stage.csv')
aux_dataset = pl.read_csv(stage5).filter((pl.col('pollutant') == 'O3') |
                                     (pl.col('pollutant') == 'PM25'))

dataset = pl.concat([dataset,aux_dataset])
dataset.write_csv(os.path.join('..','Models','results','Optimal_hyper_params.csv'))

In [32]:
import os
import ast
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

PARAMS_CSV = os.path.join('..','Models','results','Optimal_hyper_params.csv')
DATA = os.path.join('..','Data','Final_Dataset','final_dataset.parquet')
MODELS_DIR  = os.path.join('..', 'Models')
PLOTS_DIR   = os.path.join('..', 'Models','Figures')
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

dataset = pd.read_parquet(DATA)
def parse_params(s:str)->dict:
    return eval(s,{"__builtins__":None},{"np":np})

hyper_params = pd.read_csv(
    PARAMS_CSV,
    converters={'best_params': parse_params}
)

pollutants     = hyper_params['pollutant'].tolist()
pollutant_cols = [c for c in dataset.columns if c.startswith('MONTHLY')]

feature_cols = [
    "EURO_1","EURO_2","EURO_3","EURO_4","EURO_5","EURO_6","EURO_CLEAN",
    "Previous","TotalFleet","CARS_PER_CAPITA","EURO_1_PER_CAPITA",
    "EURO_2_PER_CAPITA","EURO_3_PER_CAPITA","EURO_4_PER_CAPITA",
    "EURO_5_PER_CAPITA","EURO_6_PER_CAPITA","EURO_CLEAN_PER_CAPITA",
    "Previous_PER_CAPITA"
]


for pollutant in pollutants:
    pollutant_col = next((c for c in pollutant_cols if pollutant in c), None)
    if pollutant_col is None:
        continue

    pollutant_data = dataset[dataset[pollutant_col].notna()].copy()
    
    if len(pollutant_data) < 20:
        print(f"Skipping {pollutant} due to insufficient data ({len(pollutant_data)} rows)")
        continue

    dev_data, test_data = train_test_split(
        pollutant_data, 
        test_size=0.2, 
        random_state=42,
        shuffle=True
    )

    train_data, val_data = train_test_split(
        dev_data, 
        test_size=0.1, 
        random_state=42,
        shuffle=True
    )

    print(f"{pollutant}")

    X_train, y_train = train_data[feature_cols], train_data[pollutant_col]
    X_val,   y_val   = val_data[feature_cols],   val_data[pollutant_col]
    X_test,  y_test  = test_data[feature_cols],  test_data[pollutant_col]

    lgb_train = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
    lgb_val   = lgb.Dataset(X_val,   label=y_val,   free_raw_data=False)

    best_params = (
        hyper_params.loc[hyper_params['pollutant'] == pollutant, 'best_params']
        .iloc[0]
    )
    best_params.update(dict(
        verbosity=-1, 
        metric='mape'))

    evals_result = {}

    model = lgb.train(
        best_params,
        lgb_train,
        num_boost_round=10_000,
        valid_sets=[lgb_train, lgb_val],
        valid_names=['train', 'val'],
        callbacks=[
            lgb.early_stopping(1000),
            lgb.log_evaluation(period=0),
            lgb.record_evaluation(evals_result),
        ]
    )

    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    rmse   = np.sqrt(mean_squared_error(y_test, y_pred))
    mape   = mean_absolute_percentage_error(y_test, y_pred)*100

    print(f"RMSE={rmse:.3f} | MAPE={mape:.2f}%")

    train_loss = (evals_result['train']['mape'])
    val_loss   = (evals_result['val']['mape'])

    plt.figure(figsize=(8, 4))
    plt.plot(train_loss, label='Train')
    plt.plot(val_loss,   label='Validation')
    plt.axvline(model.best_iteration, ls='--', c='k', label='Early-stop')
    plt.title(f"{pollutant.upper()} - Learning curves")
    plt.xlabel("Iteration")
    plt.ylabel("MAPE")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_DIR, f'{pollutant}_learning_curve.png'), dpi=200)
    plt.close()

    model.save_model(os.path.join(MODELS_DIR, f'{pollutant}_model.txt'),
                     num_iteration=model.best_iteration)

CO
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[31]	train's mape: 0.0900495	val's mape: 0.0914304
RMSE=0.131 | MAPE=45.14%
PM10
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[288]	train's mape: 0.138675	val's mape: 0.183267
RMSE=5.252 | MAPE=20.47%
NO2
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[348]	train's mape: 0.150861	val's mape: 0.23502
RMSE=5.300 | MAPE=23.29%
O3
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[343]	train's mape: 0.138779	val's mape: 0.207238
RMSE=10.605 | MAPE=20.73%
PM25
Training until validation scores don't improve for 1000 rounds
Did not meet early stopping. Best iteration is:
[342]	train's mape: 0.157871	val's mape: 0.23763
RMSE=2.973 | MAPE=23.44%
